# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

2023-11-03 22:58:24.625118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from time import strftime
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 7.66 ms, sys: 5.9 ms, total: 13.6 ms
Wall time: 14.6 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test_all = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 1.58 s, sys: 262 ms, total: 1.84 s
Wall time: 2.1 s


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 281 ms, sys: 54.5 ms, total: 336 ms
Wall time: 374 ms


# Exploration

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
# Numbers correspond to digits cathegories
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Pre-Processing

In [14]:
# Re-scaling values over 255 so every value in our datasets is between 0 and 1 
x_train_all, x_test = x_train_all / 255.0 , x_test / 255.0

### Convert target values to one-hot encoding

In [15]:
# converting from a sparse array to a full array 
# one-hot encoding transformation
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
# transforming datasets to one-hot encoding
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [17]:
# Dataset contains now ten columns that will refer to the number of classes in it
y_train_all.shape

(60000, 10)

In [18]:
y_test_all = np.eye(NR_CLASSES)[y_test_all]
y_test_all.shape

(10000, 10)

### Create validation dataset from training data

**Challenge:** Split the training dataset into a smaller training dataset and a validation dataset for the features and the labels. Create four arrays: x_val, y_val, x_train and, y_train from x_train_all and y_train_all. Use the validation size of 10000

In [19]:
# Using the first 10k values
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

# Using the last 50k values
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]


In [20]:
x_train.shape

(50000, 784)

# Setup Tensorflow Graph

In [21]:
# shape argument holds dimensions, this arg is provided with None since we'll be sampling different data batches
# to test model

# features
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')

# labels 
# shape arg contains 10, corresponding the total cathegories of the dataset
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [22]:
nr_epochs = 50
# scientific notation for 0.0001:
#learning_rate = 1e-4

# scientific notation for 0.001:
learning_rate = 1e-3

# defining number of neuron per layer:
n_hidden1 = 512
n_hidden2 = 64

In [23]:
# Writing a function that will define layers of the neural network

def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [24]:
## Model with no dropout layer:

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')
# 
# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')
# 
# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                      bias_dim=[NR_CLASSES], name='out')
# 
# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [25]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')


layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                     bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [26]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H:%M")}'

directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories')
    

Successfully created directories


# Loss, Optimization & Metrics

#### Defining loss function

In [27]:
# Calculating the loss from different data batches will require a mean calculus from each batch loss
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [28]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # Minimizer
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [29]:
# Comparing the predicted value against the actual value

with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [30]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [31]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [32]:
sess = tf.Session()

2023-11-03 22:58:37.933056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Setup Filewriter and Merge Summaries

In [33]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

In [34]:
# Initialise all the variables 
init = tf.global_variables_initializer()
sess.run(init)

2023-11-03 22:58:38.027364: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [35]:
# visualising weights and biases for each layer in the neural network
# w1.eval(sess)
# b1.eval(sess)
# b3.eval(sess)

# Batching Data

In [36]:
size_of_batch = 1000

In [37]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [38]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data [start:end], labels[start:end]

### Training Loop

In [39]:
for epoch in range(nr_epochs):
    
    # =============== Training Dataset ================
    
    for i in range(nr_iterations):
        # Defining batches for training
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        # feed dictionary is necessary for sess to run
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    # s stands for summary
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch{epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ============== Validation Dataset ================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
    
print("Done training!")

Epoch0 	| Training Accuracy = 0.847000002861023
Epoch1 	| Training Accuracy = 0.8610000014305115
Epoch2 	| Training Accuracy = 0.8659999966621399
Epoch3 	| Training Accuracy = 0.8669999837875366
Epoch4 	| Training Accuracy = 0.8759999871253967
Epoch5 	| Training Accuracy = 0.8790000081062317
Epoch6 	| Training Accuracy = 0.9760000109672546
Epoch7 	| Training Accuracy = 0.9779999852180481
Epoch8 	| Training Accuracy = 0.9800000190734863
Epoch9 	| Training Accuracy = 0.9789999723434448
Epoch10 	| Training Accuracy = 0.9860000014305115
Epoch11 	| Training Accuracy = 0.9869999885559082
Epoch12 	| Training Accuracy = 0.9869999885559082
Epoch13 	| Training Accuracy = 0.9869999885559082
Epoch14 	| Training Accuracy = 0.9869999885559082
Epoch15 	| Training Accuracy = 0.9890000224113464
Epoch16 	| Training Accuracy = 0.9869999885559082
Epoch17 	| Training Accuracy = 0.9890000224113464
Epoch18 	| Training Accuracy = 0.9890000224113464
Epoch19 	| Training Accuracy = 0.9900000095367432
Epoch20 	| 

# Make a prediction

In [40]:
img = Image.open('MNIST/test_img.png')
img

In [41]:
# Converting image to gray scale with:
bw = img.convert('L')

In [42]:
# Inverting image so it can be processed by our model:
img_array = np.invert(bw)

In [43]:
img_array.shape

(28, 28)

In [44]:
# Flattening the img_array to just one dimension
test_img = img_array.ravel()

In [45]:
test_img.shape

(784,)

In [46]:
# Feeding test_img to TensorFlow
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [47]:
print(f'Prediction for test image is: {prediction}')

Prediction for test image is: [2]


# Testing and Evaluation

**Challenge:** Calculate the accuracy over the test dataset(x_test, y_test). Use your Knowledgeof running session to get the accuracy. Display the accuracy as a percentage rounded to two decimal numbers

In [48]:
# accuracy can be fetched because it was defined before for our model
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test_all})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.75%


# Reset for the Next Run

In [49]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Code for 1st part of module

In [50]:
# FIRST HIDDEN LAYER:

# with tf.name_scope('initial_layer'):
# 
#     # defining initial weight
#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     # Creating a TensorFlow variable called 'weight one' that will hold all the weights in the first hidden layer
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')
#     
#     # Defining biases that will affect activation function for the neuron in the hidden layer
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')
#     
#     # tf.matmul is multiplying vectors from the first layer and its weights
#     layer1_in = tf.matmul(X, w1) + b1
#     
#     # Output for the first hidden layer and its activation function
#     layer1_out = tf.nn.relu(layer1_in)

**Challenge:** Set up the second hidden layer. This layer has 64 neurons and needs to work the output of the first hidden layer. Then set up the output layer. The output layer will use the softmax activation function

In [51]:
# SECOND HIDDEN LAYER:

# with tf.name_scope('second_layer'):
# 
#     # Weights for the second hidden layer - shape first arg gets the number of neurons from the hidden layer 1
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')
# 
#     # Biases for the second hidden layer
#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')
# 
#     # input for second hidden layer:
#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [52]:
#  FINAL - OUTPUT LAYER:

# with tf.name_scope('output_layer'):
# 
#     # Weights for the second hidden layer - shape is number of neurons from the hidden layer 2 and the cathegories
#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')
# 
#     # Biases for the second hidden layer - shape is the number of classes we want to predict
#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')
# 
#     # input for second hidden layer:
#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)